In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json

In [2]:
headers = {
            "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/85.0.4183.102 Safari/537.36', 'Accept-Language': 'en-US'#'zh-CN'
        }

In [3]:
def get_url_list(n):
    linklist=[]
    IDlist = []
    for pagenum in range(1,n):
        r = requests.get('https://store.steampowered.com/search/?ignore_preferences=1&category1=998&os=win&filter=globaltopsellers&page=%d'%pagenum,headers=headers)
        soup = BeautifulSoup(r.text, 'lxml')
        soups= soup.find_all(href=re.compile(r"https://store.steampowered.com/app/"),class_="search_result_row ds_collapse_flag")
        for i in soups:
            i = i.attrs
            i = i['href']
            link = re.search('https://store.steampowered.com/app/(\d*?)/',i).group()
            ID = re.search('https://store.steampowered.com/app/(\d*?)/(.*?)/', i).group(1)
            linklist.append(link)
            IDlist.append(ID)
        print(str(pagenum)+' pages completed, now '+str(len(linklist)))
    #print(linklist)
    return linklist,IDlist

In [4]:
def getdf(n):#turn the result to df
    linklist,IDlist = get_url_list(n)
    df = pd.DataFrame(list(zip(linklist,IDlist)),
               columns =['Link', 'ID'])
    #print(df)
    return df

In [5]:
def gamename(soup):   #name of game
    try:
        a = soup.find(class_="apphub_AppName")
        k = str(a.string)
    except:
        a = soup.find(class_="apphub_AppName")
        k = str(a.text)
    return k

In [6]:
"Racing\nAction\nMultiplayer\nCharacter Customization\nSingleplayer\nCombat Racing\nOpen World\nDriving\nAnime\nArcade\nCasual\nController\nOnline Co-Op\nSimulation\nMusic\nAdventure\nAtmospheric\nSplit Screen\nSports\nImmersive".split("\n")

['Racing',
 'Action',
 'Multiplayer',
 'Character Customization',
 'Singleplayer',
 'Combat Racing',
 'Open World',
 'Driving',
 'Anime',
 'Arcade',
 'Casual',
 'Controller',
 'Online Co-Op',
 'Simulation',
 'Music',
 'Adventure',
 'Atmospheric',
 'Split Screen',
 'Sports',
 'Immersive']

In [7]:
def taglist(soup):#tag list
    list1=[]
    a = soup.find_all(class_="app_tag")
    for i in a:
        k = str(i.string).replace('	', '').replace('\n', '').replace('\r', '')
        if k == '+':
            pass
        else:
            list1.append(k)
    list1 = str('\n'.join(list1)).split("\n")
    return list1

In [8]:
def gameprice(soup):#price
    try:
        a = soup.findAll(class_="discount_original_price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    except:
        a = soup.findAll(class_="game_purchase_price price")
        for i in a:
            if re.search('$|free', str(i),re.IGNORECASE):
                a = i
        k = str(a.string).replace('	', '').replace('\n', '').replace('\r', '').replace(' ', '')
    if(re.search('free', k,re.IGNORECASE)):
        return 0
    else:
        return float(k[1:])

In [9]:
def gamerate(soup):#get rating
    a = soup.find(class_="user_reviews_summary_row")
    k = str((a.attrs)['data-tooltip-html'])
    return float(k.split("%")[0])/100

In [10]:
def open_cache(CACHE_FILENAME):
    try:
        cache_file = open(CACHE_FILENAME, 'r')
        cache_contents = cache_file.read()
        cache_dict = json.loads(cache_contents)
        cache_file.close()
    except:
        cache_dict = []
    return cache_dict
def write_cache(game_json, CACHE_FILENAME):
    dumped_json_cache = json.dumps(game_json, indent=2)
    fw = open(CACHE_FILENAME,"w")
    fw.write(dumped_json_cache)
    fw.close()

In [11]:
a = requests.get("https://store.steampowered.com/app/1938090/", headers=headers,timeout=10)
print(a.text)


<!DOCTYPE html>
<html class=" responsive" lang="en">
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
			<meta name="viewport" content="width=device-width,initial-scale=1">
		<meta name="theme-color" content="#171a21">
		<title>Call of Duty®: Modern Warfare® II on Steam</title>
	<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon">

	
	
	<link href="https://store.cloudflare.steamstatic.com/public/shared/css/motiva_sans.css?v=2C1Oh9QFVTyK&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css" >
<link href="https://store.cloudflare.steamstatic.com/public/shared/css/shared_global.css?v=PiKnKh8sEn-U&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css" >
<link href="https://store.cloudflare.steamstatic.com/public/shared/css/buttons.css?v=hFJKQ6HV7IKT&amp;l=english&amp;_cdn=cloudflare" rel="stylesheet" type="text/css" >
<link href="https://store.cloudflare.steamstatic.com/public/css/v6/store.css?v=qccb9r09U-3n&amp;l=english&

In [12]:

game_all = {
    #"game_name":{
    #   "taglist": [],price:0,rate:0
    # }
}
tag_all = {}
price_all = {
    "free":[],
    "0-30":[],
    "30-100":[],
    "100-200":[],
    "more expensive than 200":[]
}
rate_all = {
    "0\%-20\%":[],
    "20\%-40\%":[],
    "40\%-60\%":[],
    "60\%-80\%":[],
    "80\%-100\%":[]
}
cache = open_cache("cache.json")
if len(cache) == 0:
    df = getdf(130)
    for url in df['Link']:
        if(len(game_all) >= 2000):
            break
        if(len(game_all)%100 == 0):
            print(len(game_all))
        a = requests.get(url, headers=headers,timeout=10)
        soup = BeautifulSoup(a.text, 'lxml')
        try:
            n = gamename(soup)
            tags = taglist(soup)
            p = gameprice(soup)
            rate = gamerate(soup)
            game_all[n] = {
                "taglist" : tags,
                "price":p,
                "rate":rate
            }
            for tag in tags:
                if tag in tag_all:
                    tag_all[tag].append(n)
                else:
                    tag_all[tag] = [n]
            if p == 0:
                price_all["free"].append(n)
            elif p < 30:
                price_all["0-30"].append(n)
            elif p < 100:
                price_all["30-100"].append(n)
            elif p < 200:
                price_all["100-200"].append(n)
            else:
                price_all["more expensive than 200"].append(n)
            match int(rate*10):
                case 0:
                    rate_all["0\%-20\%"].append(n)
                case 1:
                    rate_all["0\%-20\%"].append(n)
                case 2:
                    rate_all["20\%-40\%"].append(n)
                case 3:
                    rate_all["20\%-40\%"].append(n)
                case 4:
                    rate_all["40\%-60\%"].append(n)
                case 5:
                    rate_all["40\%-60\%"].append(n)
                case 6:
                    rate_all["60\%-80\%"].append(n)
                case 7:
                    rate_all["60\%-80\%"].append(n)
                case 8:
                    rate_all["80\%-100\%"].append(n)
                case 9:
                    rate_all["80\%-100\%"].append(n)
                case 10:
                    rate_all["80\%-100\%"].append(n)
        except:
            continue
    
    write_cache(game_all, "cache.json")
else:
    for n in cache:
        tags = cache[n]["taglist"]
        for tag in tags:
            if tag in tag_all:
                tag_all[tag].append(n)
            else:
                tag_all[tag] = [n]
        p = cache[n]["price"]
        if p == 0:
            price_all["free"].append(n)
        elif p < 30:
            price_all["0-30"].append(n)
        elif p < 100:
            price_all["30-100"].append(n)
        elif p < 200:
            price_all["100-200"].append(n)
        else:
            price_all["more expensive than 200"].append(n)
        rate = cache[n]["rate"]
        match int(rate*10):
            case 0:
                rate_all["0\%-20\%"].append(n)
            case 1:
                rate_all["0\%-20\%"].append(n)
            case 2:
                rate_all["20\%-40\%"].append(n)
            case 3:
                rate_all["20\%-40\%"].append(n)
            case 4:
                rate_all["40\%-60\%"].append(n)
            case 5:
                rate_all["40\%-60\%"].append(n)
            case 6:
                rate_all["60\%-80\%"].append(n)
            case 7:
                rate_all["60\%-80\%"].append(n)
            case 8:
                rate_all["80\%-100\%"].append(n)
            case 9:
                rate_all["80\%-100\%"].append(n)
            case 10:
                rate_all["80\%-100\%"].append(n)



In [13]:
while(True):
    print("Welcome to game searching")
    game = []
    while(True):
        try:
            p = float(input("Please give a price of the game that you intend, (0 indicate free game): "))
            break
        except:
            print("Please type a valid number")
    if p == 0:
        game = price_all["free"]
    elif p < 30:
        game = price_all["0-30"]
    elif p < 100:
        game = price_all["30-100"]
    elif p < 200:
        game = price_all["100-200"]
    else:
        game = price_all["more expensive than 200"]
    tags = []
    tag = input("Please give a tag that fits the game you want, i.e. FPS, you can also type \"recommend\" if unsure of the game type you want: ")
    tags.append(tag)
    while(True):
        tag = input("Please give a tag that fits the game you want, if you are finish giving tag, type \"done\": ")
        if(tag == "done"):
            break
        else:
            tags.append(tag)
    for tag in tags:
        game = list(set(tag_all[tag]) & set(game))
    print("The game we have for you is ",game)
    next_turn = input("Do you want another search?[Y\\N]: ")
    if(next_turn == "N"):
        break

    
    
    




Welcome to game searching
The game we have for you is  ['Super Mecha Champions']
Welcome to game searching
Please type a valid number
Please type a valid number
Please type a valid number
Please type a valid number
Please type a valid number
Please type a valid number
The game we have for you is  ['Super Mecha Champions']
Welcome to game searching
The game we have for you is  []
Welcome to game searching
The game we have for you is  ['Warhammer 40,000: Darktide', 'Cruelty Squad', 'Apex Legends™', 'S.T.A.L.K.E.R.: Call of Pripyat', 'Paper Dolls 2 纸人贰', "Tiny Tina's Wonderlands", 'Ready or Not', 'Holdfast: Nations At War', 'Deep Rock Galactic', 'Sniper Elite 5', 'Battlezone: Combat Commander', 'Left 4 Dead', 'Generation Zero®', 'Necromunda: Hired Gun', 'POSTAL 4: No Regerts', 'Metro Exodus', 'Aim Lab', 'Medal of Honor™: Above and Beyond', 'S.T.A.L.K.E.R.: Clear Sky', 'Destiny 2', 'Battlezone Gold Edition', 'STAR WARS™ Battlefront™ II', 'Sword Art Online: Fatal Bullet', 'Space Hulk: Death

In [14]:
#print(tag_all.keys())
#list(set(tag_all["FPS"]) & set(tag_all["Sci-fi"]))
try:
    price = float(input("hehe: "))
except:
    print("hehe")
price = price+1

hehe


NameError: name 'price' is not defined